# 0.0 Imports

In [1]:
import pandas as pd

from keras.models                    import Sequential
from keras.layers                    import Dense, Dropout
from sklearn.metrics                 import confusion_matrix
from sklearn.preprocessing           import LabelEncoder
from sklearn.model_selection         import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

2024-11-10 15:00:45.472153: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 15:00:45.474495: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 15:00:45.544851: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 15:00:45.546656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 15:00:46.888810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
spam = pd.read_csv( 'spam.csv' )
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
label_enc = LabelEncoder()
y = label_enc.fit_transform( spam['Category'] )
print( y )

[0 0 1 ... 0 0 0]


In [4]:
messages = spam['Message'].values

X_train, X_test, y_train, y_test = train_test_split( messages, y, test_size = 0.3 )
print( messages )

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 ... 'Pity, * was in mood for that. So...any other suggestions?'
 "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"
 'Rofl. Its true to its name']


In [5]:
vectorizer = CountVectorizer()
vectorizer.fit( X_train )
X_train = vectorizer.transform( X_train )
X_test = vectorizer.transform( X_test )
print( X_train.toarray() )

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
print( X_train.shape[1] )

7160


In [7]:
model = Sequential()
model.add( Dense( units=10, activation="relu", input_dim=( X_train.shape[1] ) ) )
model.add( Dropout( 0.1 ) )
model.add( Dense( units=8, activation="relu" ) )
model.add( Dropout( 0.1 ) )
model.add( Dense( units=1, activation="sigmoid" ) )

2024-11-10 15:00:48.995501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-10 15:00:48.995919: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
model.compile( loss="mean_squared_error", optimizer="adam", metrics=["accuracy"] )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 dense (Dense)               (None, 10)                71610     
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 71707 (280.11 KB)
Trainable params: 71707 (280.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.fit( X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test) )

Epoch 1/20
390/390 [==============================] - 3s 6ms/step - loss: 0.0998 - accuracy: 0.9264 - val_loss: 0.0194 - val_accuracy: 0.9827
Epoch 2/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0151 - accuracy: 0.9864 - val_loss: 0.0121 - val_accuracy: 0.9862
Epoch 3/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0080 - accuracy: 0.9910 - val_loss: 0.0116 - val_accuracy: 0.9874
Epoch 4/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0048 - accuracy: 0.9946 - val_loss: 0.0118 - val_accuracy: 0.9850
Epoch 5/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0039 - accuracy: 0.9951 - val_loss: 0.0122 - val_accuracy: 0.9844
Epoch 6/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0024 - accuracy: 0.9969 - val_loss: 0.0123 - val_accuracy: 0.9856
Epoch 7/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0019 - accuracy: 0.9972 - val_loss: 0.0126 - val_accuracy: 0.9856
Epoch 

In [10]:
loss, accuracy = model.evaluate( X_test, y_test )
print( "Loss: ", loss )
print( "Accuracy: ", accuracy )

53/53 [==============================] - 0s 2ms/step - loss: 0.0151 - accuracy: 0.9827
Loss:  0.015149851329624653
Accuracy:  0.9826555252075195


In [11]:
new_prev = model.predict( X_test )
print( new_prev )

53/53 [==============================] - 0s 2ms/step
[[3.4784858e-08]
 [2.6606845e-06]
 [2.2836708e-04]
 ...
 [2.1245200e-11]
 [9.9999917e-01]
 [9.9848032e-01]]


In [12]:
prev = ( new_prev > 0.5 )
print( prev )

[[False]
 [False]
 [False]
 ...
 [False]
 [ True]
 [ True]]


In [13]:
cm = confusion_matrix( y_test, prev )
print( cm )

[[1442    8]
 [  21  201]]
